## Libraries

In [1]:
from statsbombpy import sb
import pandas as pd

# Supress warnings from not having a full StatsBomb subscription:
import warnings
warnings.filterwarnings("ignore", message="credentials were not supplied. open data access only")

---

## Extracting Shots

All shots will be extracted from all male competitions in the StatsBomb open data, shown below.

*NOTE: 1999/2000 CL (season_id=76) is excluded as it is causing errors.*

In [2]:
sb.competitions().query("competition_gender == 'male' & season_id != 76")

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-08-17T23:51:11.837478,None,None,2023-08-17T23:51:11.837478
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
5,16,26,Europe,Champions League,male,False,False,2014/2015,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
6,16,25,Europe,Champions League,male,False,False,2013/2014,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
7,16,24,Europe,Champions League,male,False,False,2012/2013,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-07-10T13:41:45.751
8,16,23,Europe,Champions League,male,False,False,2011/2012,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
9,16,22,Europe,Champions League,male,False,False,2010/2011,2022-01-26T21:07:11.033473,2021-06-13T16:17:31.694,None,2022-01-26T21:07:11.033473


The process iterates over each match in each of these competitions, and extracts all open play shots from each match. Then, the competition and season of the events are added before adding to the overall data frame `shots`. Finally, drop all columns that only contain `NaN`.

**NOTE: This cell takes a few hours to run on my machine (see times below).**

In [3]:
%%time

shots = pd.DataFrame()

for season_index,season in sb.competitions().query("competition_gender == 'male' & season_id != 76").iterrows():
    for match_index,match in sb.matches(season_id=season.season_id, competition_id=season.competition_id).iterrows():
        match_shots = sb.events(match_id=match.match_id).query("(type == 'Shot') & (shot_type == 'Open Play')")
        match_shots["competition"] = [match.competition for i in range(match_shots.shape[0])]
        match_shots["season"] = [match.season[0:4] for i in range(match_shots.shape[0])]
        
        shots = pd.concat([shots, match_shots], ignore_index=True)
        
shots = shots.dropna(axis="columns", how="all")

CPU times: total: 3h 1min 35s
Wall time: 3h 48min 52s


### Checks

Check the added columns to see they have worked properly:

In [4]:
print(shots.competition.value_counts(), "\n\n", shots.season.value_counts())

Spain - La Liga                   19612
England - Premier League          10250
International - FIFA World Cup     2938
India - Indian Super league        2881
Europe - UEFA Euro                 1193
Europe - Champions League           397
Name: competition, dtype: int64 

 2015    18117
2021     2881
2018     2397
2020     1966
2022     1382
2017      920
2014      892
2003      848
2011      836
2016      832
2009      805
2013      778
2010      715
2008      712
2012      685
2019      680
2007      646
2006      621
2005      380
2004      178
Name: season, dtype: int64


In [4]:
print(shots.competition.value_counts(), "\n\n", shots.season.value_counts())

Spain - La Liga                                      19640
England - Premier League                             10250
Italy - Serie A                                       9429
France - Ligue 1                                      8262
Germany - 1. Bundesliga                               7364
International - FIFA World Cup                        3566
India - Indian Super league                           2881
Europe - UEFA Euro                                    1193
Europe - Champions League                              501
Europe - UEFA Europa League                             84
Spain - Copa del Rey                                    65
Argentina - Liga Profesional                            48
North and Central America - North American League       48
International - FIFA U20 World Cup                      27
Name: competition, dtype: int64 

 2015    43140
2021     2881
2018     2397
2020     1966
2022     1382
2017      920
2014      892
2003      848
2011      836
2016      832

See what the final data frame looks like:

In [5]:
shots

,duration,id,index,location,match_id,minute,off_camera,out,period,play_pattern,...,type,under_pressure,competition,season,shot_deflected,shot_open_goal,shot_redirect,shot_saved_off_target,shot_saved_to_post,shot_follows_dribble
0,0.597523,ba46e9d6-e828-4599-952c-39c1f7d22659,263,"[108.1, 31.2]",3890561,4,NaN,NaN,1,Regular Play,...,Shot,NaN,Germany - 1. Bundesliga,2015,NaN,NaN,NaN,NaN,NaN,NaN
1,0.636048,85d67225-30fb-47c8-b478-cf568941a164,353,"[110.9, 42.6]",3890561,6,NaN,NaN,1,From Throw In,...,Shot,NaN,Germany - 1. Bundesliga,2015,NaN,NaN,NaN,NaN,NaN,NaN
2,0.125672,adac17d3-5e67-4e8c-b482-4bae2f36e06e,410,"[117.9, 29.1]",3890561,8,NaN,NaN,1,From Throw In,...,Shot,True,Germany - 1. Bundesliga,2015,NaN,NaN,NaN,NaN,NaN,NaN
3,1.303079,abffd193-62bc-4c8d-8636-1e3f0f0ebbe5,624,"[101.8, 27.6]",3890561,13,NaN,NaN,1,From Counter,...,Shot,NaN,Germany - 1. Bundesliga,2015,NaN,NaN,NaN,NaN,NaN,NaN
4,0.666134,d9cea903-f92a-40e1-a393-1a849d83f157,749,"[109.3, 26.5]",3890561,17,NaN,NaN,1,Regular Play,...,Shot,NaN,Germany - 1. Bundesliga,2015,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63353,0.113466,10c4112b-ee61-4917-b238-c2da1ec32c9c,3739,"[88.4, 51.9]",3750245,103,NaN,NaN,3,Regular Play,...,Shot,NaN,Europe - UEFA Europa League,1988,NaN,NaN,NaN,NaN,NaN,NaN
63354,1.758399,c2283e99-0560-4fde-a20f-e1f3a79a67a9,4226,"[111.3, 35.7]",3750245,116,NaN,NaN,4,From Corner,...,Shot,NaN,Europe - UEFA Europa League,1988,NaN,NaN,NaN,NaN,NaN,NaN
63355,1.447182,4383adaa-ca6e-40ee-b88b-1e532eb40dd8,4239,"[89.5, 43.5]",3750245,117,NaN,NaN,4,From Corner,...,Shot,NaN,Europe - UEFA Europa League,1988,NaN,NaN,NaN,NaN,NaN,NaN
63356,1.583817,b9273b8e-d59b-41ed-8a77-a9fe56fcf85f,4298,"[106.5, 36.3]",3750245,119,NaN,NaN,4,From Free Kick,...,Shot,NaN,Europe - UEFA Europa League,1988,NaN,NaN,NaN,NaN,NaN,NaN


List the column names as they are not all included above:

In [6]:
list(shots.columns)

['duration',
 'id',
 'index',
 'location',
 'match_id',
 'minute',
 'off_camera',
 'out',
 'period',
 'play_pattern',
 'player',
 'player_id',
 'position',
 'possession',
 'possession_team',
 'possession_team_id',
 'related_events',
 'second',
 'shot_aerial_won',
 'shot_body_part',
 'shot_end_location',
 'shot_first_time',
 'shot_freeze_frame',
 'shot_key_pass_id',
 'shot_one_on_one',
 'shot_outcome',
 'shot_statsbomb_xg',
 'shot_technique',
 'shot_type',
 'team',
 'team_id',
 'timestamp',
 'type',
 'under_pressure',
 'competition',
 'season',
 'shot_deflected',
 'shot_open_goal',
 'shot_redirect',
 'shot_saved_off_target',
 'shot_saved_to_post',
 'shot_follows_dribble']

---

## Write to CSV

All checks look good, so now write data frame to CSV file.

In [8]:
shots.to_csv("statsbomb_open_shots.csv", index=False)